# PyGSTi CHP Object Test

In [1]:
from __future__ import print_function #python 2 & 3 compatibility
import pygsti
pygsti.__version__

'0.9.9.2.post715+gcde6e292.d20210315'

In [2]:
from pygsti.objects.operation import CHPOp, StochasticCHPOp
from pygsti.tools.internalgates import standard_gatenames_chp_conversions

## CHPOp

In [3]:
c = CHPOp(['h q0', 'c q1 q2'])

In [4]:
print(c)
print(c.get_chp_str(q0=0,q1=3,q2=4))

CHP operation with labels: h {q0}, c {q1} {q2}
h 0
c 3 4



In [5]:
Gx = CHPOp('Gxpi q0')
print(Gx)
print(Gx.get_chp_str(q0=1))

CHP operation with labels: h {q0}, p {q0}, p {q0}, h {q0}
h 1
p 1
p 1
h 1



In [6]:
print(CHPOp('Gc20 q0'))

CHP operation with labels: p {q0}, h {q0}, p {q0}, p {q0}, h {q0}


In [7]:
sc = StochasticCHPOp(['Gxpi q0', 'Gc20 q0'], [0.2, 0.8], seed_or_state=2021)
print(sc)
print(sc.get_chp_str(q0=0))
print(sc.get_chp_str(q0=0))
print(sc.get_chp_str(q0=0))
print(sc.get_chp_str(q0=0))

Stochastic CHP operation with probabilities and labels:
0.2: h {q0}, p {q0}, p {q0}, h {q0}
0.8: p {q0}, h {q0}, p {q0}, p {q0}, h {q0}

p 0
h 0
p 0
p 0
h 0

p 0
h 0
p 0
p 0
h 0

h 0
p 0
p 0
h 0

p 0
h 0
p 0
p 0
h 0



## CHPForwardSimulator

Trying with an Explicit Model

In [8]:
chpexe = '/Users/sserita/Documents/notebooks/pyGSTi/2021-CHP/chp'
sim = pygsti.obj.CHPForwardSimulator(chpexe, shots=100)

In [9]:
#Initialize an empty Model object
model = pygsti.objects.ExplicitOpModel(['Q0', 'Q1'], simulator=sim, evotype='chp')

#Populate the Model object with states, effects, gates,
model['rho0'] = CHPOp(['Gi Q0', 'Gi Q1'])
model['Mdefault'] = pygsti.obj.ComputationalBasisPOVM(2, 'chp')

model['Gii'] = CHPOp(['Gi Q0', 'Gi Q1'])
model['Gxi'] = CHPOp(['Gxpi Q0', 'Gi Q1'])
model['Gix'] = CHPOp(['Gi Q0', 'Gxpi Q1'])
model['Gyi'] = CHPOp(['Gypi Q0', 'Gi Q1'])

print(model)

rho0 = CHP operation with labels: 

Mdefault = Computational(Z)-basis POVM on 2 qubits and filter None


Gii = 
CHP operation with labels: 

Gxi = 
CHP operation with labels: h {Q0}, p {Q0}, p {Q0}, h {Q0}

Gix = 
CHP operation with labels: h {Q1}, p {Q1}, p {Q1}, h {Q1}

Gyi = 
CHP operation with labels: p {Q0}, p {Q0}, h {Q0}, p {Q0}, p {Q0}, h {Q0}





In [15]:
circ = pygsti.obj.Circuit(['Gix'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 1.0),
                  (('10',), 0.0),
                  (('11',), 0.0)])

In [16]:
circ = pygsti.obj.Circuit(['Gix', 'Gxi'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 0.0),
                  (('10',), 0.0),
                  (('11',), 1.0)])